In [12]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
x = tf.placeholder(tf.float32,[None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [14]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME') #padding=‘SAME’表示使用padding，不改变图片的大小
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [15]:
x_image = tf.reshape(x, [-1,28,28,1])

In [16]:
#卷积层1
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
#pooling层1
h_pool1 = max_pool_2x2(h_conv1)

In [17]:
#卷积层2
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
#pooling层2
h_pool2 = max_pool_2x2(h_conv2)

In [18]:
#全连接层
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024]) 
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)

In [19]:
#Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_dropout = tf.nn.dropout(h_fc1, keep_prob)

In [ ]:
#LSTM

In [20]:
#输入
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_dropout, W_fc2) + b_fc2

In [21]:
#训练和预测
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_, 1), tf.argmax(y_conv, 1)), tf.float32))
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(3000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict = {x: batch[0],
                                                        y_: batch[1],
                                                        keep_prob: 1.})
            print('setp {},the train accuracy: {}'.format(i, train_accuracy))
        train_step.run(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 0.5})
    test_accuracy = accuracy.eval(feed_dict = {x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.})
    print('the test accuracy :{}'.format(test_accuracy))
    saver = tf.train.Saver()
    path = saver.save(sess, './my_net/mnist_deep.ckpt')
    print('save path: {}'.format(path))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

setp 0,the train accuracy: 0.07999999821186066
setp 100,the train accuracy: 0.8799999952316284
setp 200,the train accuracy: 0.8600000143051147
setp 300,the train accuracy: 0.9399999976158142
setp 400,the train accuracy: 0.9599999785423279
setp 500,the train accuracy: 0.8399999737739563
setp 600,the train accuracy: 0.9800000190734863
setp 700,the train accuracy: 0.9599999785423279
setp 800,the train accuracy: 0.9599999785423279
setp 900,the train accuracy: 0.9599999785423279
setp 1000,the train accuracy: 1.0
setp 1100,the train accuracy: 1.0
setp 1200,the train accuracy: 0.9399999976158142
setp 1300,the train accuracy: 0.9399999976158142
setp 1400,the train accuracy: 0.9800000190734863
setp 1500,the train accuracy: 1.0
setp 1600,the train accuracy: 0.9599999785423279
setp 1700,the train accuracy: 0.